<a href="https://colab.research.google.com/github/ngngocsonan2610/note/blob/master/1%20Data%20Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sources

In [0]:
%%html
<iframe src="https://stackedit.io/app#" width="1000" height="600"></iframe>

- [**A Comprehensive Guide to Data Exploration**](https://www.analyticsvidhya.com/blog/2016/01/guide-data-exploration/)
- [Data Analytics Collection - Hasbrain](https://note.hasbrain.com/5c875913b40548001824aa67?target=path)


# 1.EDA & Plotting

## Resume & Phân tích df
Descriptive statistics are used to describe the basic features of the data in a study. They provide simple summaries about the sample and the measures. Together with simple graphics analysis, they form the basis of virtually every quantitative analysis of data.
- Phân tích về min max, mean, median, mode, std
- Phân tích về loại của feature (continous/categorical, numeric/text)
- Đếm giá trị unique ở features/ giá trị missing 

**Sources**:
- [Đọc thêm 1](https://towardsdatascience.com/understanding-descriptive-statistics-c9c2b0641291)



In [0]:
# This function assits to expand Exploratory Data Analysis (EDA) is an open-ended 
#process where we calculate statistics and make figures to find trends, anomalies,
#patterns, or relationships within the data. The goal of EDA is to learn what our
#data can tell us. It generally starts out with a high level overview, 
#then narrows in to specific areas as we find intriguing areas of the data. 

def des_stat_analyze(df_input):
    # check number of rows, cols
    no_rows = df_input.shape[0]
    no_cols = df_input.shape[1]
    print("No. observations:", no_rows )
    print("No. features:", no_cols )
  
    # checking type of features
    name = []
    cols_type = []
    for n,t in df_input.dtypes.iteritems():
        name.append(n)
        cols_type.append(t)

    # checking distinction (unique values) of features
    ls_unique = []
    for cname in df_input.columns:
        try:
            nunique = df_input[cname].nunique()
            pct_unique = nunique*100.0/ no_rows
            ls_unique.append("{} ({:0.2f}%)".format(nunique, pct_unique))
        except:
            ls_unique.append("{} ({:0.2f}%)".format(0,0))
            continue

    # checking missing values of features
    ls_miss = []
    for cname in df_input.columns:
        try:
            nmiss = df_input[cname].isnull().sum()
            pct_miss = nmiss*100.0/ no_rows
            ls_miss.append("{} ({:0.2f}%)".format(nmiss, pct_miss))
        except:
            ls_miss.append("{} ({:0.2f}%)".format(0,0))
            continue 
      
    # checking zeros
    ls_zeros = []
    for cname in df_input.columns:
        try:
            nzeros = (df_input[cname] == 0).sum()
            pct_zeros = nzeros * 100.0/ no_rows
            ls_zeros.append("{} ({:0.2f}%)".fornat(nzeros, pct_zeros))
        except:
            ls_zeros.append("{} ({:0.2f}%)".format(0,0))
            continue
      
    # checking negative values
    ls_neg = []
    for cname in df_input.columns:
        try:
            nneg = (df_input[cname].astype("float")<0).sum()
            pct_neg =nneg * 100.0 / no_rows
            ls_neg.append("{} ({:0.2f}%)".format(nneg, pct_neg))
        except:
            ls_neg.append("{} ({:0.2f}%)".format(0,0))
            continue
      
    # extracting the output
    data = {
      "name": name,
      "col_type": cols_type,
      "n_unique": ls_unique,
      "n_miss": ls_miss,
      "n_zeros":ls_zeros,
      "n_neg":ls_neg      
    }
  
    # statistical info
    df_stats = df_input.describe().transpose()
    ex_stats = pd.concat([df_input.median(),df_input.kurtosis(),df_input.skew()], axis = 1)
    ex_stats = ex_stats.rename(columns={0:"median",1:"kurtosis",2:"skew"})
    df_stats = df_stats.merge(ex_stats, left_index=True, right_index=True)
    #ls_stats = []
    for stat in df_stats.columns:
        data[stat] = []
        for cname in df_input.columns:
            try:
                data[stat].append(df_stats.loc[cname, stat])
            except:
                data[stat].append("NaN")
        
    # take samples
    df_sample = df_input.sample(frac = .5).head().transpose()
    df_sample.columns = ["sample_{}".format(i) for i in range(5)]
  
    # repair the output
    col_ordered = ["name","col_type","count","n_unique","n_miss","n_zeros","n_neg",
                "25%","50%","75%","max","min","mean","median","std","kurtosis","skew"]
    df_data = pd.DataFrame(data, columns = col_ordered).set_index("name")
    df_data = pd.concat([df_data, df_sample], axis = 1)

    return df_data   

## Correlation

Xem correlation của các features với nhau: 
- 2 features có chỉ số correlation dương, có nghĩa khi A tăng thì B tăng (if A increase, B will increase)
- 2 features có chỉ số correlation âm, có nghĩa khi A tăng thì B giảm (if A increase, B will decrease)
- - 2 features có chỉ số correlation cao, có nghĩa là 2 features này có khả năng thay thế nhau trong model (replaces)

Sources:
-  [Covariance vs Correlation](https://forum.machinelearningcoban.com/t/covariance-va-correlation/767)

In [0]:
def plot_corr(df):
    correlations = df.corr()

    # Using seaborn package
    # Generate a mask for the upper triangle
    mask = np.zeros_like(correlations, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(16, 16))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(260, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(correlations, mask=mask, cmap=cmap, vmin = -1, vmax= 1 , center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.yticks(rotation=0)

In [0]:
def find_topcorr_bycol(col):
    df_corr = df.corr().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
    df_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}
                , inplace=True)     
    return df_corr[df_corr["Feature 1"]==col]

## Plot continous / categorical features




In [0]:

"""Plotting the continous features :    
1. A box plot (or box-and-whisker plot) shows the distribution of quantitative data 
in a way that facilitates comparisons between variables.
2. Distribution graph :to check the linearity of the variables and look 
for skewness of features."""
def plot_continous(cols, df, target=None, info = False, corr = False, drop_3Qplot=False):
    # Using boxplot to analyze the continous feature
    if corr == True:
      df_corr = df.corr().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
      df_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}
                , inplace=True)      

    for col in cols:
        print(f'\n >Column name: {col}')
        plt.figure(figsize=(15,8))
        
        if corr == True:
          print("Top 5 positive correlated w/ featuers \n{}".format(
              df_corr[(df_corr["Feature 1"]==col) & (df_corr["Feature 2"]!=col)].head()))
          print("Top 5 negative correlated w/ featuers \n{}".format(
              df_corr[(df_corr["Feature 1"]==col) & (df_corr["Feature 2"]!=col)].tail()))
        if info == True:

          print("max: {}, min: {}, mean: {}, std: {}, median: {}".format(
              df[col].max(),df[col].min(),
              round(df[col].mean(),3),round(df[col].std(),3),
              df[col].median()))
          print(f"Quantiles | {col}\n",df[col].quantile([.01, .025, .1, .25, .5, .75, .9, .975, .99]))
          print("No unique: {}, No null: {}".format(df[col].nunique(),df[col].isnull().sum()))
          if df[col].nunique() < 10 :
            print(f"Value counts in {col} \n",format(df[col].value_counts()))

        # plot  
        plt.subplot(2, 2, 1)
        if target == None:
            fig = sns.boxplot(col, whis=1.5, data=df)
            fig.legend()
        else:
            fig = sns.boxplot(x=target, y=col, whis=1.5, data=df)
            fig.legend()
        # which defined as the proportion of the IQR past the low and high quartiles to extend the plot whiskers 
        # or interquartile range (IQR)
        # therefore, maximum = Q3 + 1.5*IQR , min = Q1 - 1.5*IQR
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1    
        
        plt.subplot(2, 2, 2)
        fig = sns.distplot(df[col].dropna())#.hist(bins=20)
        fig.set_ylabel('Volumn')
        fig.set_xlabel(col)      
        
        print('No data out of range between Q1 : {} and Q3 : {} = {}' .format(Q1,Q3,((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))).sum()))

        if drop_3Qplot==True:          
          plt.subplot(2, 2, 3)
          tmp = df[(df[col] >= Q1)&(df[col] <= Q3)]#[col]
          if target == None:
              fig = sns.boxplot(col, whis=1.5, data=tmp)
              fig.legend()
          else:
              fig = sns.boxplot(x=target, y=col, whis=1.5, data=tmp)
              fig.legend()
          
          plt.subplot(2, 2, 4)
          fig = sns.distplot(tmp[col])#.dropna())#.hist(bins=20)
          fig.set_ylabel('Volumn')
          fig.set_xlabel(col)   

        plt.show() 

In [0]:
def plot_categorical(cols, target, df):
  for col in cols:
    if target == None:
      sns.countplot(x=col, data=df[[col]] ,  palette="Reds_d")      
    else:
      print('Column name: %s' %col)
      sns.countplot(x=col, hue=target, data=df[[col,target]], palette="Reds_d")
      #sns.barplot(x=col, y=target, data=df[[col,target]]
      #        , palette="Reds_d", estimator = sum)
      plt.legend()
    plt.xticks(rotation=90)#-60    
    plt.show()

# 2.Cleaning Data


## Outliers

***Check outliers by plotting***

[estimation cdf - plot](https://www.jddata22.com/home//plotting-an-empirical-cdf-in-python)
  - check outlier by std and mean
  
[boxplot & quanlite](https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51)
  - check outlier by 
    - IQR: interquantile range = Q3 - Q1
    - “maximum”: Q3 + 1.5*IQR
    - “minimum”: Q1 -1.5*IQR

**Source**
- [Three ways to detect outliers](http://colingorrie.github.io/outlier-detection.html)
- [How to Use Statistics to Identify Outliers in Data](https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/)
- [5 Ways to Find Outliers in Your Data](https://statisticsbyjim.com/basics/outliers/)

In [0]:
def CalcOutliers(df_num): 
    '''
    Find Outliers for continous cols
    Tìm outliner dựa trên std, và mean

    Sau khi phân tích quantile và mean, std
    có thể biết outlier nằm ngoài vùng nào:
    vd: cut = std * 3
    '''
    # calculating mean and std of the array
    data_mean, data_std = np.mean(df_num), np.std(df_num)

    # seting the cut line to both higher and lower values
    # You can change this value
    cut = data_std * 3

    #Calculating the higher and lower cut values
    lower, upper = data_mean - cut, data_mean + cut

    # creating an array of lower, higher and total outlier values 
    outliers_lower = [x for x in df_num if x < lower]
    outliers_higher = [x for x in df_num if x > upper]
    outliers_total = [x for x in df_num if x < lower or x > upper]

    # array without outlier values
    outliers_removed = [x for x in df_num if x > lower and x < upper]
    
    print('Identified lowest outliers: %d' % len(outliers_lower)) # printing total number of values in lower cut of outliers
    print('Identified upper outliers: %d' % len(outliers_higher)) # printing total number of values in higher cut of outliers
    print('Total outlier observations: %d' % len(outliers_total)) # printing total number of values outliers of both sides
    print('Non-outlier observations: %d' % len(outliers_removed)) # printing total number of non outlier values
    print("Total percentual of Outliers: ", round((len(outliers_total) / len(outliers_removed) )*100, 4)) # Percentual of outliers in points
    
    return

In [0]:
'''
Find Outliers for continous cols
Tìm outliner dựa trên std, và mean

Sau khi phân tích quantile và mean, std
có thể biết outlier nằm ngoài vùng nào:
vd: cut = std * 3
'''

def CalcOutliers(df_num): 

    # calculating mean and std of the array
    data_mean, data_std = np.mean(df_num), np.std(df_num)

    # seting the cut line to both higher and lower values
    # You can change this value
    cut = data_std * 3

    #Calculating the higher and lower cut values
    lower, upper = data_mean - cut, data_mean + cut

    # creating an array of lower, higher and total outlier values 
    outliers_lower = [x for x in df_num if x < lower]
    outliers_higher = [x for x in df_num if x > upper]
    outliers_total = [x for x in df_num if x < lower or x > upper]

    # array without outlier values
    outliers_removed = [x for x in df_num if x > lower and x < upper]
    
    print('Identified lowest outliers: %d' % len(outliers_lower)) # printing total number of values in lower cut of outliers
    print('Identified upper outliers: %d' % len(outliers_higher)) # printing total number of values in higher cut of outliers
    print('Total outlier observations: %d' % len(outliers_total)) # printing total number of values outliers of both sides
    print('Non-outlier observations: %d' % len(outliers_removed)) # printing total number of non outlier values
    print("Total percentual of Outliers: ", round((len(outliers_total) / len(outliers_removed) )*100, 4)) # Percentual of outliers in points
    
    return

## High correlation features
Có thể xóa bỏ bớt các cột có high correlate với nhau vì: 
- các cột có ý nghĩa như nhau với model
- có cơ hội học được từ các cột khác
- **quan trọng trong việc train NN** vì giảm được khối lượng dữ liệu

In [0]:
# drop columns with correlation greater than 0.9
corr_matrix = df.drop(columns="isMale").corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
print("high correlation features", to_drop)
df.drop(columns=to_drop, inplace=True)

# 3.Statistical Checking
Các kỹ thuật khác trong việc kiểm tra về mặt thống kê

Source:
- [Statistical Learning oxh](https://ongxuanhong.wordpress.com/category/kien-thuc/statistical-inference/)
- [Video về Statistic](https://www.youtube.com/user/drnguyenvtuan/videos)

## Bayesian Statistic
- [How Bayesian statistics convinced me to hit the gym](https://towardsdatascience.com/how-bayesian-statistics-convinced-me-to-hit-the-gym-fa737b0a7ac)
- [Think you need to learn Bayesian Analysis? Read this first](https://peadarcoyle.com/2019/01/01/think-you-need-to-learn-bayesian-analysis-read-this-first/?fbclid=IwAR1KGEuQNfzzf0BexNampJsyZ3zNgcfzDOgNMsPvppjElAu4qOOPF8AwC7g)




# 4.Before fitting model

- Used in:
  - Kaggle:[ Fraud Detection](https://www.kaggle.com/sonannguyenngoc/fraud-pca/edit/run/18581236) 
 

## Missing value 
- fill na : https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779


In [0]:
# count null or nan
df['Count_miss'] = df[col].isnull().sum(axis=1)

df['nulls1'] = df.isna().sum(axis=1)

In [0]:
'''
Tìm các bất ổn về missing value và các cột có bật ổn về số lượng giá trị
'''
many_null_cols = [col for col in df_train.columns if df_train[col].isnull().sum() / df_train.shape[0] > 0.9]
many_null_cols_test = [col for col in df_test.columns if df_test[col].isnull().sum() / df_test.shape[0] > 0.9]


big_top_value_cols = ([col for col in train.columns if 
                       df_train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9])
big_top_value_cols_test = ([col for col in test.columns if 
                            df_test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9])
one_value_cols = [col for col in df.columns if df[col].nunique() <= 1]
one_value_cols_test = [col for col in df_test.columns if df_test[col].nunique() <= 1]
one_value_cols == one_value_cols_test

cols_to_drop = list(set(many_null_cols + many_null_cols_test + 
                        big_top_value_cols + big_top_value_cols_test + 
                        one_value_cols + one_value_cols_test))
cols_to_drop.remove('isFraud')
len(cols_to_drop)

df = df.drop(cols_to_drop, axis=1)



In [0]:
# Tìm na/nan/null trong từng cột
# Hiện thị: Số na, %na, type
# GPreda, missing data
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [0]:
'''
1 số cách fill NA
interpolate from categorical features
'''
def df_processing(df):
    df = df[df['MEMBER_GENDER'].notnull()]
    df['MEMBER_OCCUPATION_CD'] = df['MEMBER_OCCUPATION_CD'].fillna(7)
    df['MEMBER_OCCUPATION_CD'] = df['MEMBER_OCCUPATION_CD'].replace({4:3,5:3,7:3,6:4})
    df['MEMBER_MARITAL_STATUS'] = df['MEMBER_MARITAL_STATUS'].fillna('UNK')
    df['MEMBER_MARITAL_STATUS'] = df['MEMBER_MARITAL_STATUS'].replace({'S':'S-W-D','W':'S-W-D','D':'S-W-D'})

    df = df[df['PAYMENT_MODE'] != 'SINGLE-PREMIUM']

    df = df[df['ANNUAL_FEES'] > 0]
    df = df[df['ANNUAL_FEES'] < df['MEMBER_ANNUAL_INCOME']]
    df = df[df['ANNUAL_FEES'] < 1000000]
    df['MEMBER_ANNUAL_INCOME'] = df.groupby(['MEMBER_OCCUPATION_CD','MEMBER_MARITAL_STATUS','MEMBER_GENDER'])
                                  \ ['MEMBER_ANNUAL_INCOME'].apply(lambda x: x.fillna(x.median()))
    df = df[df['MEMBER_ANNUAL_INCOME'] < 9000000]
    
    # special kids
    df = df[df["MEMBER_AGE_AT_ISSUE"] > 12]
    
    df["PAYMENT_MODE_NUM"] = df["PAYMENT_MODE"].replace({'ANNUAL':1,'SEMI-ANNUAL':1/2,
                                                        'QUARTERLY':1/4,'MONTHLY':1/12
                                                        })
    
    df['MEMBERSHIP_TERM_YEARS_BIN'] = df.apply(lambda df:mem_term_lab(df), axis = 1)
    # age group
    x = pd.qcut(df["MEMBER_AGE_AT_ISSUE"].to_list(),q = 8)
    x.categories = ["age_g1","age_g2","age_g3","age_g4","age_g5","age_g6","age_g7","age_g8"]
    df["AGE_GROUP"] = x.to_list()
    
    # manipuation agent
    aggent_code_churn = df.groupby(['AGENT_CODE']).agg({
        "ANNUAL_FEES":'mean',
        "MEMBER_ANNUAL_INCOME":'mean',
    }).reset_index()
    x = pd.qcut(aggent_code_churn["ANNUAL_FEES"].to_list(),q = 6,duplicates='drop')
    x.categories = [1,2,3,4,5]
    aggent_code_churn["AGENT_RANK_FEES"] = x.to_list()
    x = pd.qcut(aggent_code_churn["MEMBER_ANNUAL_INCOME"].to_list(),q = 6,duplicates='drop')
    x.categories = [1,2,3,4,5,6]
    aggent_code_churn["AGENT_RANK_INCOMES"] = x.to_list()
    aggent_code_churn.drop(columns= ["ANNUAL_FEES","MEMBER_ANNUAL_INCOME"], inplace=True)

    df = pd.merge(df,aggent_code_churn, on = "AGENT_CODE")
    
    return df
df = df_processing(df)

## Reduce memory

In [0]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

## Transformation Scaling/Standardize

## Reduce dimension checking

In [0]:
#scaling only on tranform data
df_V = df_V[c_V_trans] 
df_V_test = df_V_test[c_V_trans]

#standardize scaled
scaler = StandardScaler().fit(df_V)
df_V_scaled = scaler.transform(df_V)
print('Scaled mean df_V',df_V_scaled[:,0].mean())  # zero (or very close)
print('Scaled std df_V',df_V_scaled[:,0].std()) 
scaler = StandardScaler().fit(df_V_test)
df_V_test_scaled = scaler.transform(df_V_test)
print('Scaled mean df_V',df_V_test_scaled[:,0].mean())  # zero (or very close)
print('Scaled std df_V',df_V_test_scaled[:,0].std()) 

# plot cumulative explained variance
# pca = PCA().fit(df_V_scaled)
# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# #plt.xlim(0,7,1)
# plt.xlabel('Number of components')
# plt.ylabel('Cumulative explained variance')


# Setup Principal component analysis
pca = PCA(n_components=125) 
#pca = PCA(n_components=0.96)  #v2: n_components=0.95
df_V_pca = pca.fit_transform(df_V_scaled)
np.save('df_V_pca_v3.npy',df_V_pca)
#df_V_pca.to_csv('df_V_pca', sep='\t')
df_V_test_pca = pca.fit_transform(df_V_test_scaled)
np.save('df_V_test_pca_v3.npy',df_V_test_pca)


In [0]:
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization

'''
Scatter plot giữa 3 cột bất kỳ trong dữ liệu, được hue = TARGET
cho cái nhìn về dữ liệu, xem liệu nó có khả năng phân tách không
'''

trace1 = go.Scatter3d(x = churn["MEMBER_ANNUAL_INCOME"],
                      y = churn["ANNUAL_FEES"],
                      z = churn["MEMBERSHIP_TERM_YEARS"],
                      mode = "markers",
                      name = "Churn customers",
                      text = "Id : " + churn["MEMBERSHIP_NUMBER"],
                      marker = dict(size = 1,color = "red")
                     )
trace2 = go.Scatter3d(x = not_churn["MEMBER_ANNUAL_INCOME"],
                      y = not_churn["ANNUAL_FEES"],
                      z = not_churn["MEMBERSHIP_TERM_YEARS"],
                      name = "Non churn customers",
                      text = "Id : " + not_churn["MEMBERSHIP_NUMBER"],
                      mode = "markers",
                      marker = dict(size = 1,color= "green")
                     )

layout = go.Layout(dict(title = "Monthly charges,total charges & tenure in customer attrition",
                        scene = dict(camera = dict(up=dict(x= 0 , y=0, z=0),
                                                   center=dict(x=0, y=0, z=0),
                                                   eye=dict(x=1.25, y=1.25, z=1.25)),
                                     xaxis  = dict(title = "annual incomes",
                                                   gridcolor='rgb(255, 255, 255)',
                                                   zerolinecolor='rgb(255, 255, 255)',
                                                   showbackground=True,
                                                   backgroundcolor='rgb(230, 230,230)'),
                                     yaxis  = dict(title = "annual fees",
                                                   gridcolor='rgb(255, 255, 255)',
                                                   zerolinecolor='rgb(255, 255, 255)',
                                                   showbackground=True,
                                                   backgroundcolor='rgb(230, 230,230)'
                                                  ),
                                     zaxis  = dict(title = "term years",
                                                   gridcolor='rgb(255, 255, 255)',
                                                   zerolinecolor='rgb(255, 255, 255)',
                                                   showbackground=True,
                                                   backgroundcolor='rgb(230, 230,230)'
                                                  )
                                    ),
                        height = 700,
                       )
                  )
                  

data = [trace1,trace2]
fig  = go.Figure(data = data,layout = layout)
py.iplot(fig)

In [0]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)

X = tmp[[i for i in tmp.columns if i not in Id_col + target_col]]
Y = tmp[target_col + Id_col]

principal_components = pca.fit_transform(X)
pca_data = pd.DataFrame(principal_components,columns = ["PC1","PC2"])
pca_data = pca_data.merge(Y,left_index=True,right_index=True,how="left")
pca_data["CHURN"] = pca_data["CHURN"].replace({1:"CANCELLED",0:"INFORCE"})

def pca_scatter(target,color) :
    tracer = go.Scatter(x = pca_data[pca_data["CHURN"] == target]["PC1"] ,
                        y = pca_data[pca_data["CHURN"] == target]["PC2"],
                        name = target,mode = "markers",
                        marker = dict(color = color,
                                      line = dict(width = .5),
                                      symbol =  "diamond-open"),
                        text = ("Customer Id : " + 
                                pca_data[pca_data["CHURN"] == target]['MEMBERSHIP_NUMBER'])
                       )
    return tracer

layout = go.Layout(dict(title = "Visualising data with principal components",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "principal component 1",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "principal component 2",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        height = 600
                       )
                  )
trace1 = pca_scatter("CANCELLED",'red')
trace2 = pca_scatter("INFORCE",'royalblue')
data = [trace2,trace1]
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

# New



# New



# New

